In [1]:
#--Librerias--#
import pandas as pd 
import a_funciones as funciones  
from sklearn.model_selection import RandomizedSearchCV # búsqueda aleatoria de hiperparámetros 
import joblib  ### para guardar y cargar modelos
from sklearn.preprocessing import StandardScaler ## escalar variables 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Regresión logística
from sklearn.ensemble import RandomForestClassifier  # Clasificador bosques aleatoriost 
from sklearn.tree import DecisionTreeClassifier # Arboles de decision 
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler ## escalar variable
from sklearn.ensemble import GradientBoostingClassifier #Modelo de clasificacion
from sklearn.metrics import confusion_matrix #### Matriz de confusion 
import seaborn as sns #####Graficos
from sklearn.metrics import accuracy_score #### medir precision del modelo 

In [2]:
###----Lectura de datos----###
df = pd.read_csv("https://raw.githubusercontent.com/andressj1/Predicci-n-precio-de-m-viles-/refs/heads/main/Datos/df_fin", sep=",")

# Dummies

In [3]:
# Variables categoricas
list_cat = ['mobile_wt_cat','ram_cat','tamaño']

In [4]:
# Conversion; categoricas a dummies
df = pd.get_dummies(df, columns=list_cat)

In [5]:
# Separar var_objetivo de las demas
y=df.price_range
X1= df.loc[:,~df.columns.isin(['price_range'])]

In [6]:
# ¿duda codigo?
scaler=StandardScaler()
scaler.fit(X1)
X2=scaler.transform(X1)
X=pd.DataFrame(X2,columns=X1.columns)

# Selección de variables

In [7]:
### Selección de modelos 
m_rl = LogisticRegression()
m_rf = RandomForestClassifier()
m_add = DecisionTreeClassifier()

In [8]:
### Lista de modelos
modelos = list([m_rl, m_rf, m_add])

In [9]:
var_names=funciones.sel_variables(modelos,X,y,threshold="2.5*mean")
var_names.shape

(4,)

In [10]:
X2=X[var_names] ### matriz con variables seleccionadas
X2.info()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958 entries, 0 to 1957
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1958 non-null   float64
 1   px_width       1958 non-null   float64
 2   ram_cat_2GB    1958 non-null   float64
 3   ram_cat_4GB    1958 non-null   float64
dtypes: float64(4)
memory usage: 61.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958 entries, 0 to 1957
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   battery_power          1958 non-null   float64
 1   blue                   1958 non-null   float64
 2   dual_sim               1958 non-null   float64
 3   fc                     1958 non-null   float64
 4   four_g                 1958 non-null   float64
 5   int_memory             1958 non-null   float64
 6   m_dep                  1958 non-null   float64
 7   n_cores  

# Modelo ganador

In [11]:
## base con todas las variables
rmse_df = funciones.medir_modelos(modelos, "f1",X,y,10)
## base con variables seleccionadas
rmse_varsel = funciones.medir_modelos(modelos,"f1",X2,y,10)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py", line 1271, in f1_score
    return fbeta_score(
           ^^^^^^^^^^^^
  File "c:\Program

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements